In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

print("Files loaded!")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

train_df = pd.read_csv('/kaggle/input/mlp-en-cifar-100/cifar100_train.csv')


In [ ]:
train_df.head()

In [ ]:
base_dir = '/kaggle/input/mlp-en-cifar-100/'

In [ ]:
import matplotlib.pyplot as plt
from skimage.io import imread
%matplotlib inline
img = imread(base_dir+train_df['Image'][0])
plt.imshow(img)
print(img.shape)

In [ ]:
x_train_and_val = np.zeros((50000,32,32,3),dtype='uint8')
for i, imgname in enumerate(train_df['Image'].values):
    x_train_and_val[i,] = np.array(imread(base_dir+imgname))


In [ ]:
# Agarramos cada clase también
y_train_and_val = train_df['Label'].values
print(y_train_and_val.shape)
print(y_train_and_val[2])

In [ ]:
# Grafiquemos una imagen para ver que se esten agarrando bien
plt.imshow(x_train_and_val[2,])
x_train_and_val[2,].shape

In [ ]:
# Ploteo de 36 imagenes del dataset de train
def plot_images(x_train_and_val, y_train_and_val, N, n_rows=3, n_cols=12):
    fig = plt.figure(figsize=(20, 5))
    for i in range(N):
        ax = fig.add_subplot(n_rows, n_cols, i + 1, xticks=[], yticks=[])
        ax.set_xlabel(y_train_and_val[i])
        plt.imshow(x_train_and_val[i,])
plot_images(x_train_and_val, y_train_and_val, 36)


In [ ]:
# Divido en train y validacion
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_and_val, y_train_and_val,
                                                  test_size=0.05, 
                                                  random_state = 42,
                                                  stratify=y_train_and_val
                                                 )


In [ ]:
# Agrego un flip a mi dataset de train
x_augmented_fliped = []
y_augmented_fliped = []
from PIL import Image, ImageOps
for i in range(len(x_train)):
    image = x_train[i]
    pil_img = Image.fromarray(image)
    pil = pil_img.transpose(Image.FLIP_LEFT_RIGHT)
    image = np.asarray(pil)
    x_augmented_fliped.append(image)
    y_augmented_fliped.append(y_train[i])

In [ ]:
x_train = np.asarray([*x_train, *x_augmented_fliped])
y_train = np.asarray([*y_train, *y_augmented_fliped])

In [ ]:
plt.imshow(x_train[40000-5])

In [ ]:
plt.imshow(x_train[-5])

In [ ]:
# Realizo el One hot encoding de mis clases
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc_val = OneHotEncoder(handle_unknown='ignore')
y_categorical = enc.fit_transform(y_train.reshape(-1, 1))
y_val_categorical = enc_val.fit_transform(y_val.reshape(-1, 1))
print(y_train)
print(y_train.reshape(-1, 1))
print(y_categorical)
print(y_categorical[0,])
print(y_categorical[0,].shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, normalization, Dropout, Convolution2D, Conv2DTranspose, MaxPooling2D, Activation

# define the model
model = Sequential()
model.add(Convolution2D(64, (4, 4),padding="same", input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2DTranspose(64, (4, 4), padding="same", input_shape=x_train.shape[1:]))
model.add(Convolution2D(32, (4, 4), padding="same", input_shape=x_train.shape[1:]))
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(normalization.BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(normalization.BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(normalization.BatchNormalization())
model.add(Dense(100, activation='softmax'))
model.summary()

In [ ]:
# Compilo modelo, asigno optimizador, funcion de costo, metrica a utilizar
# categorical_crossentropy no se cambia
# sparse categorical cross entropy ?
# Optimizador se cambia! Se puede instanciar un optimizador y variarle los parametros
# Metrics es para que el modelo me vaya guardando los datos
import keras
optimizer = keras.optimizers.SGD(
    learning_rate=0.003, momentum=0.9
)
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'],
)

In [ ]:
from IPython.display import clear_output
class PlotLosses(keras.callbacks.Callback):
    def __init__(self, plot_interval=1, evaluate_interval=10, x_val=None, y_val_categorical=None):
        self.plot_interval = plot_interval
        self.evaluate_interval = evaluate_interval
        self.x_val = x_val
        self.y_val_categorical = y_val_categorical
        #self.model = model
    
    def on_train_begin(self, logs={}):
        print('Begin training')
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.logs = []
    
    def on_epoch_end(self, epoch, logs={}):
        if self.evaluate_interval is None:
            self.logs.append(logs)
            self.x.append(self.i)
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            self.acc.append(logs.get('acc'))
            self.val_acc.append(logs.get('val_acc'))
            self.i += 1
        
        if (epoch%self.plot_interval==0):
            clear_output(wait=True)
            f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(20,5))
            ax1.plot(self.x, self.losses, label="loss")
            ax1.plot(self.x, self.val_losses, label="val_loss")
            ax1.legend()

            ax2.plot(self.x, self.acc, label="acc")
            ax2.plot(self.x, self.val_acc, label="val_acc")
            ax2.legend()
            plt.show();
        #score = self.model.evaluate(x_test, y_test_categorical, verbose=0)
        
        #print("accuracy: ", score[1])
    
    def on_batch_end(self, batch, logs={}):
        if self.evaluate_interval is not None:
            if (batch%self.evaluate_interval==0):
                self.i += 1
                self.logs.append(logs)
                self.x.append(self.i)
                self.losses.append(logs.get('loss'))
                self.acc.append(logs.get('acc'))

                if self.x_val is not None:
                    score = self.model.evaluate(self.x_val, self.y_val_categorical, verbose=0)
                    self.val_losses.append(score[0])
                    self.val_acc.append(score[1])

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
checkpointer = ModelCheckpoint(filepath='model_best.hdf5', monitor='val_accuracy',verbose=0, save_best_only=True)
plot_losses = PlotLosses(plot_interval=5, evaluate_interval=None, x_val=x_val, y_val_categorical=y_val_categorical)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)  
hist = model.fit(
    x_train, # cargo x traing
    y_categorical, # cargo y encoded
    batch_size=512, # asigno un batch size
    epochs=250,
    verbose=2,
    shuffle=True,
    validation_data=(x_val, y_val_categorical),
    callbacks=[plot_losses, checkpointer]
)

In [ ]:
# Cargo pesos optimos
model.load_weights('model_best.hdf5')
model.weights

In [ ]:
model.evaluate(x_val, y_val_categorical)

In [ ]:
test_df = pd.read_csv(base_dir + 'cifar100_test.csv')
# Treat the test data in the same way as training data. In this case, pull same columns.
x_test = np.zeros((10000, 32, 32, 3),dtype='uint8')
for i,imgname in enumerate(test_df['Image'].values):
    img = imread(base_dir + imgname)
    x_test[i, :, :, :] = img

test_df.head()


In [ ]:
test_df.head()


In [ ]:
# Use the model to make predictions
predicted_classes = np.argmax(model.predict(x_test),axis=1)
print(predicted_classes.shape)
print(predicted_classes[5])

In [ ]:
predicted_classes

In [ ]:
y_labels = enc.get_feature_names()
y_labels

In [ ]:
y_labels = [y.split('x0_')[-1] for y in enc.get_feature_names()]
y_labels

In [ ]:
y_labels[predicted_classes[6]]

In [ ]:
y_predictions = []
for i in range(len(predicted_classes)):
    y_predictions.append(y_labels[predicted_classes[i]])

In [ ]:
my_submission = pd.DataFrame(
    {
    'Image': test_df['Image'],
    'Label': y_predictions
    }
)
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [ ]:
from IPython.display import FileLink
FileLink(r'submission.csv')